In [1]:
import data_request_api
import json
from collections import defaultdict
import datetime

In [2]:
import data_request_api.content.dreq_content as dc
import data_request_api.query.dreq_query as dq

use_dreq_version = 'v1.2.1'
content = dc.load(use_dreq_version)#, offline=True)

# TABLE CONSTRUCTION

In [3]:
all_var_info = dq.get_variables_metadata(content, use_dreq_version)

In [4]:
# all_var_info = json.loads(json.dumps(all_var_info))

In [5]:
def entry_to_cmorvar(entry):
    simple_entries = ["cell_measures", "cell_methods", "comment", "out_name", "positive", "units", 
                      "long_name", "standard_name", "branded_variable_name"]
    cmorvar = {i:entry[i] for i in simple_entries}
    cmorvar['brand_description'] = '?'
    cmorvar['dimensions'] = entry['dimensions'].split()
    cmorvar['realm'] = [entry['modeling_realm']]
    cmorvar['long_name'] = entry['long_name']

    return cmorvar
    

In [6]:
cmorvars = [entry_to_cmorvar(i) for i in all_var_info.values()]

In [7]:


tables = defaultdict(dict)

In [8]:
for v in all_var_info.values():
    cmorvar = entry_to_cmorvar(v)
    tables[cmorvar["realm"][0]][cmorvar["branded_variable_name"]] = cmorvar

In [9]:
GENERIC_LEVELS = {
    'atmos': "alevel alevhalf", 
    'land': "", 
    'ocean': "olevel olevhalf", 
    'aerosol': "alevel alevhalf", 
    'atmosChem': "alevel alevhalf",
    'landIce': "", 
    'ocnBgchem': "olevel olevhalf", 
    'seaIce': "olevel olevhalf"
}

TIMESTAMP = datetime.datetime.today().strftime("%Y-%m-%d %T")

def write_table(tables):
    for realm in tables:
        template = {
            "Header":{
                "Conventions": "CMIP7-TESTING",
                "checksum": "unknown",
                "cmor_version": "3.10",
                "generic_levels": GENERIC_LEVELS[realm],
                "int_missing_value": "-999",
                "missing_value": "1e20",
                "ok_max_mean_abs": "",
                "ok_min_mean_abs": "",
                "positive": "",
                "product": "model-output",
                "realm": realm,
                "table_date": TIMESTAMP,
                "table_id": realm,
                "type": "real",
                "valid_max": "",
                "valid_min": ""
            },
            "variable_entry":{}
        }
        for v, cmorvar in tables[realm].items():
            
            for i in ['branded_variable_name', 'realm']:
                if i in cmorvar:
                    del cmorvar[i]
            template['variable_entry'][v] = cmorvar
        with open('development-tables/CMIP7_{}.json'.format(realm), 'w') as fh:
            json.dump(template, fh, indent=4, sort_keys=True)
    

In [10]:
write_table(tables)

# COORDINATE FILE CONSTRUCTION

In [11]:
coords = content['Data Request v1.2.1']['Coordinates and Dimensions']

In [12]:
def coord_to_entry(coord):
    name = coord.name
    entry = {}
    mapping = {
        'axis': 'axis_flag',
        'bounds_values': 'bounds_scalar', 
        'climatology': 'climatology_flag',
        'formula': None,  # does not exist
        'generic_level_name': None,  # does not exist
        'long_name': 'title',
        'must_have_bounds': 'bounds_flag',
        'out_name': 'output_name',
        'positive': 'positive_direction',
        'requested': 'requested_values',
        'requested_bounds': 'requested_bounds',
        'standard_name': 'cf_standard_name',
        'stored_direction': 'stored_direction',
        'tolerance': 'tolerance',
        'type': 'type',
        'units': 'units',
        'valid_max': 'maximum_valid_value',
        'valid_min': 'minimum_valid_value',
        'value': 'value_scalar_or_string',
        'z_bounds_factors': None,
        'z_factors': None,
    }
    
    for cmor_key, dr_key in mapping.items():
        if dr_key is None:
            value = ''
        else:
            value = getattr(coord, dr_key, '')
        if isinstance(value, str):
            value = value.replace(',', '')
        entry[cmor_key] = value
    
    # deal with yes no fields
    if entry['must_have_bounds']:
        entry['must_have_bounds'] = "yes"
    else:
        entry['must_have_bounds'] = "no"
    # deal with lists
    if entry['requested']:
        try:
            entry['requested'] = ['{:.1f}'.format(float(i)) for i in entry['requested'].split()]
        except ValueError:
            entry['requested'] = ''
    if entry['requested_bounds']:
        entry['requested_bounds'] = ['{:.1f}'.format(float(i)) for i in entry['requested_bounds'].split()]

    # convert numbers to strings (even if they are zero)
    for i in ['tolerance', 'valid_max', 'valid_min']:
        if entry[i] != "":
            entry[i] = str(entry[i])

    return name, entry    

     

Construct coordinates dictionary

In [13]:
coordinates = {}

for coord in coords.records.values():
    name, entry = coord_to_entry(coord)
    coordinates[name] = entry


In [14]:
reference = json.load(open('reference/MIP_coordinate.json'))

In [15]:
keys_to_drop_from_dr = [
    'alevel',
    'alevhalf',
    'olevel',
    'olevhalf',
]

# keys_that_cause_cmor_failure = [
#     'xant',
#     'yant',
#     'xgre',
#     'ygre'
# ]

# for k in keys_to_drop_from_dr + keys_that_cause_cmor_failure:
#     if k in coordinates:
#         del coordinates[k]

keys_to_import = [
    'alternate_hybrid_sigma',
    'alternate_hybrid_sigma_half',
    'depth_coord',
    'depth_coord_half',
    'hybrid_height',
    'hybrid_height_half',
    'ocean_sigma',
    'ocean_sigma_half',
    'ocean_sigma_z',
    'ocean_sigma_z_half',
    'standard_hybrid_sigma',
    'standard_hybrid_sigma_half',
    'standard_sigma',
    'standard_sigma_half']

for k in keys_to_import:
    coordinates[k] = reference['axis_entry'][k]


In [16]:
with open('development-tables/CMIP7_{}.json'.format('coordinate'), 'w') as fh:
    json.dump({'axis_entry': coordinates}, fh, indent=4, sort_keys=True)

# ancillary files
(From mip-cmor-tables)

In [17]:
! cp reference/MIP_formula_terms.json development-tables/CMIP7_formula_terms.json
! cp reference/MIP_grids.json development-tables/CMIP7_grids.json

# CVs

(from CMOR TestTables directory)

In [18]:
! cp reference/CMIP7_CV.json development-tables/CMIP7_CV.json